In [1]:
import os
import json

import requests
import csv
import dateutil
import time
import datetime as dt

In [10]:
import pandas as pd

In [2]:
with open('credentials.json', 'r') as json_file:
    cred = json.load(json_file)

In [21]:
search_url = "https://api.twitter.com/2/tweets/search/recent"

In [ ]:
query_params = {'query': '(#climatechange OR climate change) -is:retweet lang:en',
                    'end_time': '2022-08-01T00:00:00Z',
                    'max_results': max_results,
                    'tweet.fields': 'created_at,public_metrics',
                    'place.fields': 'full_name,id,country,country_code,geo,name,place_type',
                    'next_token': {}}

In [4]:
def bearer_oauth(r):
    r.headers['Authorization'] = f'Bearer {cred["BEARER_TOKEN"]}'
    r.headers['User-Agent'] = "v2RecentSearchPython"
    return r

In [5]:
def connect_to_endpoint(url, params, next_token=None):
    params['next_token'] = next_token
    response = requests.get(url, auth=bearer_oauth, params=params)
    print(response.status_code)
    if response.status_code != 200:
        raise Exception(response.status_code, response.text)
    return response.json()

In [20]:
def create_query_params(query, end_date, max_results = 10):

    #change params based on the endpoint you are using
    query_params = {'query': query,
                    'end_time': end_date,
                    'max_results': max_results,
                    'tweet.fields': 'created_at,public_metrics',
                    'place.fields': 'full_name,id,country,country_code,geo,name,place_type',
                    'next_token': {}}
    
    return query_params

In [15]:
def append_to_csv(json_response, fileName):

    #A counter variable
    counter = 0

    #Open OR create the target CSV file
    csvFile = open(fileName, "a", newline="", encoding='utf-8')
    csvWriter = csv.writer(csvFile)

    #Loop through each tweet
    for tweet in json_response['data']:
        
        # We will create a variable for each since some of the keys might not exist for some tweets
        # So we will account for that

        # 1. Tweet ID
        tweet_id = tweet['id']

        # 2. Time created
        created_at = dateutil.parser.parse(tweet['created_at'])
        
        # 3. Tweet text
        text = tweet['text']

        # 4. Tweet metrics
        retweet_count = tweet['public_metrics']['retweet_count']
        reply_count = tweet['public_metrics']['reply_count']
        like_count = tweet['public_metrics']['like_count']
        quote_count = tweet['public_metrics']['quote_count']
        
        # Assemble all data in a list
        res = [tweet_id, created_at, text, like_count, quote_count, reply_count, retweet_count]
        
        # Append the result to the CSV file
        csvWriter.writerow(res)
        counter += 1

    # When done, close the CSV file
    csvFile.close()

    # Print the number of tweets for this iteration
    print("# of Tweets added from this response: ", counter) 

In [16]:
def calc_dates_list(start_datetime, end_datetime, delta_hours):
    dates_to_extract = list(pd.date_range(
        start=start_datetime,
        end=end_datetime - dt.timedelta(seconds=1),
        freq=dt.timedelta(
            hours=delta_hours)).to_pydatetime())
    return dates_to_extract

In [17]:
# start_date = '2022-07-14T00:00:00Z'
# end_date = '2022-07-14T03:00:00Z'

start_date = '2022-07-26T01:00:00Z'
end_date = '2022-08-01T12:00:00Z'
date_format = "%Y-%m-%dT%H:%M:%SZ"

dates_list = calc_dates_list(
    dt.datetime.strptime(start_date, date_format),
    dt.datetime.strptime(end_date, date_format), 
    delta_hours=1
)

In [18]:
# json_response = connect_to_endpoint(search_url, query_params)
# json_response['data'][0]

In [19]:
#Inputs for tweets
keyword = '(#climatechange OR climate change) -is:retweet lang:en'
max_results = 100

#Total number of tweets we collected from the loop
total_tweets = 0

# Create file
csvFile = open("data.csv", "a", newline="", encoding='utf-8')
csvWriter = csv.writer(csvFile)

#Create headers for the data you want to save, in this example, we only want save these columns in our dataset
csv_headers = ['tweet_id', 'created_at', 'tweet', 'like_count', 'quote_count', 'reply_count', 'retweet_count']
csvWriter.writerow(csv_headers)
csvFile.close()

for end_date in dates_list:
    
    print(f'extracting data for {end_date}')

    # Inputs
    count = 0 # Counting tweets per time period
    max_count = 1000 # Max tweets per time period
    flag = True
    next_token = None
    
    # Check if flag is true
    while flag:
        # Check if max_count reached
        if count >= max_count:
            break
        print("-------------------")
        print("Token: ", next_token)
        
        query_params = create_query_params(keyword, end_date.strftime(date_format), max_results)
        
        json_response = connect_to_endpoint(search_url, query_params, next_token)
        result_count = json_response['meta']['result_count']

        if 'next_token' in json_response['meta']:
            # Save the token to use for next call
            next_token = json_response['meta']['next_token']
            print("Next Token: ", next_token)
            if result_count is not None and result_count > 0 and next_token is not None:
                # print("Start Date: ", start_list[i])
                append_to_csv(json_response, "data.csv")
                count += result_count
                total_tweets += result_count
                print("Total # of Tweets added: ", total_tweets)
                print("-------------------")
                time.sleep(5)                
        # If no next token exists
        else:
            if result_count is not None and result_count > 0:
                print("-------------------")
                # print("Start Date: ", start_list[i])
                append_to_csv(json_response, "data.csv")
                count += result_count
                total_tweets += result_count
                print("Total # of Tweets added: ", total_tweets)
                print("-------------------")
                time.sleep(5)
            
            #Since this is the final request, turn flag to false to move to the next time period.
            flag = False
            next_token = None
        time.sleep(30)
print("Total number of results: ", total_tweets)

extracting data for 2022-07-26 01:00:00
-------------------
Token:  None
400


Exception: (400, '{"errors":[{"parameters":{"query":["from:TwitterDev","(#climatechange OR climate change) -is:retweet lang:en"]},"message":"Duplicate parameters are not allowed: the `query` query parameter"}],"title":"Invalid Request","detail":"One or more parameters to your request was invalid.","type":"https://api.twitter.com/2/problems/invalid-request"}')

In [ ]:
# print(json.dumps(json_response, indent=4, sort_keys=True))